In [2]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from os import listdir
from PIL import Image
import glob
from pickle import dump, load
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.applications import imagenet_utils

Using TensorFlow backend.


In [3]:
def load_doc(filename):
    file = open(filename,'r')
    text = file.read()
    return text

In [4]:
# load a pre-defined list of photo identifiers
def load_set(filename):
    doc = load_doc(filename)
    dataset = list()
    # process line by line
    for line in doc.split('\n'):
        # skip empty lines
        if len(line) < 1:
            continue
        # get the image identifier
        identifier = line.split('.')[0]
        dataset.append(identifier)
    return (dataset)

In [5]:
# load training dataset (6K)
train_text_filename = 'Flickr8k_text/Flickr_8k.trainImages.txt'
train = load_set(train_text_filename)
print('Train-Dataset: %d' % len(train))

Train-Dataset: 6000


In [6]:
train

['2513260012_03d33305cf',
 '2903617548_d3e38d7f88',
 '3338291921_fe7ae0c8f8',
 '488416045_1c6d903fe0',
 '2644326817_8f45080b87',
 '218342358_1755a9cce1',
 '2501968935_02f2cd8079',
 '2699342860_5288e203ea',
 '2638369467_8fc251595b',
 '2926786902_815a99a154',
 '2851304910_b5721199bc',
 '3423802527_94bd2b23b0',
 '3356369156_074750c6cc',
 '2294598473_40637b5c04',
 '1191338263_a4fa073154',
 '2380765956_6313d8cae3',
 '3197891333_b1b0fd1702',
 '3119887967_271a097464',
 '2276499757_b44dc6f8ce',
 '2506892928_7e79bec613',
 '2187222896_c206d63396',
 '2826769554_85c90864c9',
 '3097196395_ec06075389',
 '3603116579_4a28a932e2',
 '3339263085_6db9fd0981',
 '2532262109_87429a2cae',
 '2076906555_c20dc082db',
 '2502007071_82a8c639cf',
 '3113769557_9edbb8275c',
 '3325974730_3ee192e4ff',
 '1655781989_b15ab4cbff',
 '1662261486_db967930de',
 '2410562803_56ec09f41c',
 '2469498117_b4543e1460',
 '69710415_5c2bfb1058',
 '3414734842_beb543f400',
 '3006217970_90b42e6b27',
 '2192411521_9c7e488c5e',
 '3535879138_928

In [7]:
# loading full dataset image name  
image_names = []
images_directory= 'Flickr8k_Dataset'
for name in listdir(images_directory):
    image_names.append(images_directory+'/'+name)
print(image_names)

['Flickr8k_Dataset/359173181_a75c950aeb.jpg', 'Flickr8k_Dataset/3282897060_8c584e2ce8.jpg', 'Flickr8k_Dataset/3362592729_893e26b806.jpg', 'Flickr8k_Dataset/1052358063_eae6744153.jpg', 'Flickr8k_Dataset/2219805677_7b7cc188c7.jpg', 'Flickr8k_Dataset/1423126855_6cd2a3956c.jpg', 'Flickr8k_Dataset/3497236690_a48bf7ac42.jpg', 'Flickr8k_Dataset/801607443_f15956d1ce.jpg', 'Flickr8k_Dataset/2998945968_86f236d1e8.jpg', 'Flickr8k_Dataset/3595408539_a7d8aabc24.jpg', 'Flickr8k_Dataset/3562903245_85071bb5f9.jpg', 'Flickr8k_Dataset/3535084928_858544f49a.jpg', 'Flickr8k_Dataset/367964525_b1528ac6e4.jpg', 'Flickr8k_Dataset/3044536048_e615466e7f.jpg', 'Flickr8k_Dataset/2873188959_ff023defa9.jpg', 'Flickr8k_Dataset/3329254388_27017bab30.jpg', 'Flickr8k_Dataset/2052202553_373dad145b.jpg', 'Flickr8k_Dataset/2518853257_02f30e282e.jpg', 'Flickr8k_Dataset/3309578722_1765d7d1af.jpg', 'Flickr8k_Dataset/2648165716_02e2e74fd6.jpg', 'Flickr8k_Dataset/1287920676_d21a0b289b.jpg', 'Flickr8k_Dataset/3080891382_edf83dd

In [9]:
train_names = []
# check and load training images
for i in image_names:
    i=i.split('/')
    
    if i[-1].split('.')[0] in train:
        train_names.append('/'.join(i))
print(len(train_names))
print('\n','\n',train_names)

6000

 
 ['Flickr8k_Dataset/359173181_a75c950aeb.jpg', 'Flickr8k_Dataset/3282897060_8c584e2ce8.jpg', 'Flickr8k_Dataset/3362592729_893e26b806.jpg', 'Flickr8k_Dataset/1052358063_eae6744153.jpg', 'Flickr8k_Dataset/2219805677_7b7cc188c7.jpg', 'Flickr8k_Dataset/1423126855_6cd2a3956c.jpg', 'Flickr8k_Dataset/3497236690_a48bf7ac42.jpg', 'Flickr8k_Dataset/2998945968_86f236d1e8.jpg', 'Flickr8k_Dataset/3595408539_a7d8aabc24.jpg', 'Flickr8k_Dataset/3562903245_85071bb5f9.jpg', 'Flickr8k_Dataset/3535084928_858544f49a.jpg', 'Flickr8k_Dataset/367964525_b1528ac6e4.jpg', 'Flickr8k_Dataset/2873188959_ff023defa9.jpg', 'Flickr8k_Dataset/3329254388_27017bab30.jpg', 'Flickr8k_Dataset/2052202553_373dad145b.jpg', 'Flickr8k_Dataset/2518853257_02f30e282e.jpg', 'Flickr8k_Dataset/3309578722_1765d7d1af.jpg', 'Flickr8k_Dataset/1287920676_d21a0b289b.jpg', 'Flickr8k_Dataset/3080891382_edf83dde18.jpg', 'Flickr8k_Dataset/2934022873_3fdd69aee4.jpg', 'Flickr8k_Dataset/2101808682_0d66ef4a08.jpg', 'Flickr8k_Dataset/23640961

In [22]:
#load test dataset images
# load training dataset (6K)
test_text_filename = 'Flickr8k_text/Flickr_8k.testImages.txt'
test = load_set(test_text_filename)
print('Test-Dataset: %d' % len(test),'\n')

test_names = []
# check and load testing images
for i in image_names:
    i=i.split('/')
    
    if i[-1].split('.')[0] in test:
        test_names.append('/'.join(i))
        
print(len(test_names))        
print('\n','\n',test_names)

Test-Dataset: 1000 

1000

 
 ['Flickr8k_Dataset/801607443_f15956d1ce.jpg', 'Flickr8k_Dataset/3044536048_e615466e7f.jpg', 'Flickr8k_Dataset/2648165716_02e2e74fd6.jpg', 'Flickr8k_Dataset/3286822339_5535af6b93.jpg', 'Flickr8k_Dataset/3157847991_463e006a28.jpg', 'Flickr8k_Dataset/1472230829_803818a383.jpg', 'Flickr8k_Dataset/2623930900_b9df917b82.jpg', 'Flickr8k_Dataset/2197275664_fabcf3424b.jpg', 'Flickr8k_Dataset/247704641_d883902277.jpg', 'Flickr8k_Dataset/1836335410_de8313a64e.jpg', 'Flickr8k_Dataset/880220939_0ef1c37f1f.jpg', 'Flickr8k_Dataset/370614351_98b8a166b9.jpg', 'Flickr8k_Dataset/339350939_6643bfb270.jpg', 'Flickr8k_Dataset/2021613437_d99731f986.jpg', 'Flickr8k_Dataset/3686924335_3c51e8834a.jpg', 'Flickr8k_Dataset/3609032038_005c789f64.jpg', 'Flickr8k_Dataset/3003691049_f4363c2d5c.jpg', 'Flickr8k_Dataset/2054869561_ff723e9eab.jpg', 'Flickr8k_Dataset/3642220260_3aa8a52670.jpg', 'Flickr8k_Dataset/3191135894_2b4bdabb6d.jpg', 'Flickr8k_Dataset/2049051050_20359a434a.jpg', 'Flickr8

In [14]:
def preprocess(image_path):
    # Convert all the images to size 299x299 as expected by the inception v3 model
    img = load_img(image_path, target_size=(299, 299))
    # Convert PIL image to numpy array of 3-dimensions
    x = img_to_array(img)
    # Add one more dimension
    x = np.expand_dims(x, axis=0)
    # preprocess the images using preprocess_input() from inception module
    x = preprocess_input(x)
    return x

In [15]:
# Load the inception v3 model
model = InceptionV3(weights='imagenet')

In [16]:
# Create a new model, by removing the last layer (output layer) from the inception v3
model_new = Model(model.input, model.layers[-2].output)

In [17]:
# Function to encode a given image into a vector of size (2048, )
def encode(image):
    image = preprocess(image) # preprocess the image
    fea_vec = model_new.predict(image,verbose=0) # Get the encoding vector for the image
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1]) # reshape from (1, 2048) to (2048, )
    return fea_vec

In [26]:
# Call the funtion to encode all the train images
start = time()
a=1
encoding_train = {}
for img in train_names:
    encoding_train[img.split('/')[-1]] = encode(img)
    print(a)
    a=a+1
print("Time taken in seconds =", time()-start)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061


3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701


5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340


In [28]:
# Save the bottleneck train features to disk
with open("encoded_train_images.pkl", "wb") as encoded_pickle:
   dump(encoding_train, encoded_pickle)

In [23]:
# Call the funtion to encode all the test images - Execute this only once
start = time()
encoding_test = {}
a=1
for img in test_names:
    encoding_test[img.split('/')[-1]] = encode(img)
    print(a)
    a=a+1
print("Time taken in seconds =", time()-start)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [29]:
# Save the bottleneck test features to disk
with open("encoded_test_images.pkl", "wb") as encoded_pickle:
   dump(encoding_test, encoded_pickle)